<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-00-5171310e77f2'
PROJECT = 'qwiklabs-gcp-00-5171310e77f2'
REGION = 'us-west1-b'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [10]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery 
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use ABS(MOD()) on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [16]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-7170969733900686954,331274
1,5896567601480310696,342825
2,-774501970389208065,349134
3,-5742197815970064689,359684
4,454960867574323744,346515


In [32]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13365 examples in the train dataset and 3198 in the eval dataset


In [26]:
sampling_query = "SELECT COUNT(weight_pounds) FROM (" + query + ") WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0001"
print(sampling_query)
traindf.shape

SELECT COUNT(weight_pounds) FROM (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
) WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0001


(13325, 6)

In [24]:
df = bigquery.Client().query(sampling_query).to_dataframe()
df

,f0_
0,2630


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [34]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.779215,False,20,1,39.0,3095933535584005890
1,9.356418,False,28,1,38.0,3095933535584005890
2,5.749656,False,18,1,36.0,3095933535584005890
3,8.113011,False,28,1,40.0,3095933535584005890
4,7.308324,False,25,1,39.0,3095933535584005890


In [35]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13358.000000,13365.000000,13365.000000,13245.000000,1.336500e+04
mean,7.241510,27.311111,1.032847,38.597206,2.804504e+17
std,1.322390,6.153980,0.188843,2.548623,5.209513e+18
min,0.513677,12.000000,1.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-3.340563e+18
50%,7.322654,27.000000,1.000000,39.000000,-3.280124e+17
75%,8.062305,32.000000,1.000000,40.000000,4.331750e+18
max,12.125424,51.000000,5.000000,47.000000,8.599690e+18


In [36]:
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
twins_etc
#traindf['plurality'].replace(twins_etc, inplace=True)
#traindf.head()

{1: 'Single(1)',
 2: 'Twins(2)',
 3: 'Triplets(3)',
 4: 'Quadruplets(4)',
 5: 'Quintuplets(5)'}

In [38]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [39]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.779215,False,20,Single(1),39.0,3095933535584005890
1,9.356418,False,28,Single(1),38.0,3095933535584005890
2,5.749656,False,18,Single(1),36.0,3095933535584005890
3,8.113011,False,28,Single(1),40.0,3095933535584005890
4,7.308324,False,25,Single(1),39.0,3095933535584005890


In [40]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13360,6.812284,Unknown,20,Single(1),39.0,-774501970389208065
13361,6.062712,Unknown,24,Single(1),39.0,-774501970389208065
13362,7.120931,Unknown,35,Single(1),40.0,-774501970389208065
13363,5.562263,Unknown,18,Single(1),41.0,-774501970389208065
13364,6.124442,Unknown,19,Single(1),39.0,-774501970389208065


In [43]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26482.000000,26482.000000,26482.000000,2.648200e+04
mean,7.244203,27.315611,38.600257,2.799604e+17
std,1.319512,6.145376,2.536887,5.205869e+18
min,0.513677,12.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,38.000000,-3.340563e+18
50%,7.325961,27.000000,39.000000,-3.280124e+17
75%,8.062305,32.000000,40.000000,4.331750e+18
max,12.125424,50.000000,47.000000,8.599690e+18


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License